In [114]:
import pandas as pd
import numpy as np

from sklearn import preprocessing
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier

from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV

from sklearn.metrics import accuracy_score, recall_score, precision_score

import pickle

import warnings

warnings.filterwarnings('ignore')

# Load data

### Initialize path constant

In [115]:
DATA_PATH = '../data'
RAW_DATA_PATH = '{}/raw'.format(DATA_PATH)
PROCESSED_DATA_PATH = '{}/processed'.format(DATA_PATH)

MODEL_PATH = '../models'

### Read processed data from CSV

In [116]:
# data = pd.read_csv('{}/processed_data.csv'.format(PROCESSED_DATA_PATH))
data = pd.read_csv('{}/_processed_data.csv'.format(PROCESSED_DATA_PATH))
X = data.drop(['heart_disease_diagnosis'], axis=1).values
y = data['heart_disease_diagnosis'].values

# Cross Validation

### CV Function

In [117]:
def get_kfold():
    return KFold(n_splits=5, shuffle=True, random_state=1)

def print_cv_result(model, X, y):
    accuracy_scores = []
    precision_scores = []
    recall_scores = []
    
    kfold = get_kfold()
    
    for train_idx, validation_idx in kfold.split(X, y):
        X_train = X[train_idx]
        y_train = y[train_idx]
        X_validation = X[validation_idx]
        y_validation = y[validation_idx]

        model.fit(X_train, y_train)

        prediction = model.predict(X_validation)

        accuracy = accuracy_score(y_validation, prediction)
        precision = precision_score(y_validation, prediction, average='macro')
        recall = recall_score(y_validation, prediction, average='macro')

        accuracy_scores.append(accuracy)
        precision_scores.append(precision)
        recall_scores.append(recall)
    
    print('--- Validation Metrics ---')
    print('Accuracy  = {:.3f}'.format(np.mean(accuracy_scores)))
    print('Precision = {:.3f}'.format(np.mean(precision_scores)))
    print('Recall    = {:.3f}'.format(np.mean(recall_scores)))

## Models

### Naive Bayes

In [118]:
model_name = 'Naive Bayes'
nb_model = GaussianNB()

print('=== {} ===\n'.format(model_name))
print_cv_result(nb_model, X, y)

=== Naive Bayes ===

--- Validation Metrics ---
Accuracy  = 0.546
Precision = 0.353
Recall    = 0.370


### KNN

In K-Nearest Neighbors algorithm, it's really important to scale the features first (feature scaling).
Since the range of values of raw data varies widely, in K-Nearest Neighbors algoritm, objective functions will not work properly without normalization. For example, the majority of classifiers calculate the distance between two points by the Euclidean distance. If one of the features has a broad range of values, the distance will be governed by this particular feature. Therefore, the range of all features should be normalized so that each feature contributes approximately proportionately to the final distance.

In [119]:
model_name = 'K-Nearest Neighbor'
knn_model = KNeighborsClassifier()
X_Scaled = preprocessing.scale(X)

print('=== {} ===\n'.format(model_name))
print_cv_result(knn_model, X_Scaled, y)

=== K-Nearest Neighbor ===

--- Validation Metrics ---
Accuracy  = 0.548
Precision = 0.298
Recall    = 0.314


### Decision Tree

In [120]:
model_name = 'Decision Tree'
dtc_model = DecisionTreeClassifier(criterion='entropy', random_state=1)

print('=== {} ===\n'.format(model_name))
print_cv_result(dtc_model, X, y)

=== Decision Tree ===

--- Validation Metrics ---
Accuracy  = 0.458
Precision = 0.313
Recall    = 0.313


### ANN

In [121]:
model_name = 'ANN'
ann_model = MLPClassifier(random_state=1, activation='logistic')
X_Scaled = preprocessing.scale(X)

print('=== {} ===\n'.format(model_name))
print_cv_result(ann_model, X_Scaled, y)

=== ANN ===

--- Validation Metrics ---
Accuracy  = 0.582
Precision = 0.347
Recall    = 0.351


# Tune Best Base Model

In [124]:
param_grid = {
    'hidden_layer_sizes': [(50,), (100,), (50, 50), (100, 100)],
    'solver': ['lbfgs', 'sgd', 'adam'],
    'alpha': [1e-4, 1e-3, 1e-2, 1e-1],
    'learning_rate': ['constant', 'invscaling', 'adaptive']
}

best_model = GridSearchCV(MLPClassifier(random_state=1, activation='logistic'),
                          param_grid,
                          cv=get_kfold(),
                          scoring='accuracy',
                          verbose=20)

best_model.fit(X_Scaled, y)

Fitting 5 folds for each of 144 candidates, totalling 720 fits
[CV] alpha=0.0001, hidden_layer_sizes=(50,), learning_rate=constant, solver=lbfgs 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  alpha=0.0001, hidden_layer_sizes=(50,), learning_rate=constant, solver=lbfgs, score=0.46153846153846156, total=   0.2s
[CV] alpha=0.0001, hidden_layer_sizes=(50,), learning_rate=constant, solver=lbfgs 
[CV]  alpha=0.0001, hidden_layer_sizes=(50,), learning_rate=constant, solver=lbfgs, score=0.5384615384615384, total=   0.2s
[CV] alpha=0.0001, hidden_layer_sizes=(50,), learning_rate=constant, solver=lbfgs 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.4s remaining:    0.0s


[CV]  alpha=0.0001, hidden_layer_sizes=(50,), learning_rate=constant, solver=lbfgs, score=0.46153846153846156, total=   0.2s
[CV] alpha=0.0001, hidden_layer_sizes=(50,), learning_rate=constant, solver=lbfgs 
[CV]  alpha=0.0001, hidden_layer_sizes=(50,), learning_rate=constant, solver=lbfgs, score=0.46794871794871795, total=   0.2s
[CV] alpha=0.0001, hidden_layer_sizes=(50,), learning_rate=constant, solver=lbfgs 


...

[CV]  alpha=0.1, hidden_layer_sizes=(100,), learning_rate=adaptive, solver=sgd, score=0.5512820512820513, total=   0.6s
[CV] alpha=0.1, hidden_layer_sizes=(100,), learning_rate=adaptive, solver=sgd 
[CV]  alpha=0.1, hidden_layer_sizes=(100,), learning_rate=adaptive, solver=sgd, score=0.5833333333333334, total=   0.6s
[CV] alpha=0.1, hidden_layer_sizes=(100,), learning_rate=adaptive, solver=sgd 
[CV]  alpha=0.1, hidden_layer_sizes=(100,), learning_rate=adaptive, solver=sgd, score=0.5448717948717948, total=   0.6s
[CV] alpha=0.1, hidden_layer_sizes=(100,), learning_rate=adaptive, solver=sgd 
[CV]  alpha=0.1, hidden_layer_sizes=(100,), learning_rate=adaptive, solver=sgd, score=0.6089743589743589, total=   0.6s
[CV] alpha=0.1, hidden_layer_sizes=(100,), learning_rate=adaptive, solver=sgd 
[CV]  alpha=0.1, hidden_layer_sizes=(100,), learning_rate=adaptive, solver=sgd, score=0.5290322580645161, total=   0.6s
[CV] alpha=0.1, hidden_layer_sizes=(100,), learning_rate=adaptive, solver=adam 
[CV]

[CV]  alpha=0.1, hidden_layer_sizes=(50, 50), learning_rate=adaptive, solver=lbfgs, score=0.5192307692307693, total=   0.5s
[CV] alpha=0.1, hidden_layer_sizes=(50, 50), learning_rate=adaptive, solver=lbfgs 
[CV]  alpha=0.1, hidden_layer_sizes=(50, 50), learning_rate=adaptive, solver=lbfgs, score=0.5705128205128205, total=   0.5s
[CV] alpha=0.1, hidden_layer_sizes=(50, 50), learning_rate=adaptive, solver=lbfgs 
[CV]  alpha=0.1, hidden_layer_sizes=(50, 50), learning_rate=adaptive, solver=lbfgs, score=0.5, total=   0.5s
[CV] alpha=0.1, hidden_layer_sizes=(50, 50), learning_rate=adaptive, solver=lbfgs 
[CV]  alpha=0.1, hidden_layer_sizes=(50, 50), learning_rate=adaptive, solver=lbfgs, score=0.4645161290322581, total=   0.5s
[CV] alpha=0.1, hidden_layer_sizes=(50, 50), learning_rate=adaptive, solver=sgd 
[CV]  alpha=0.1, hidden_layer_sizes=(50, 50), learning_rate=adaptive, solver=sgd, score=0.4358974358974359, total=   0.7s
[CV] alpha=0.1, hidden_layer_sizes=(50, 50), learning_rate=adaptive

[CV]  alpha=0.1, hidden_layer_sizes=(100, 100), learning_rate=invscaling, solver=adam, score=0.5961538461538461, total=   1.2s
[CV] alpha=0.1, hidden_layer_sizes=(100, 100), learning_rate=invscaling, solver=adam 
[CV]  alpha=0.1, hidden_layer_sizes=(100, 100), learning_rate=invscaling, solver=adam, score=0.5576923076923077, total=   1.5s
[CV] alpha=0.1, hidden_layer_sizes=(100, 100), learning_rate=invscaling, solver=adam 
[CV]  alpha=0.1, hidden_layer_sizes=(100, 100), learning_rate=invscaling, solver=adam, score=0.5641025641025641, total=   2.1s
[CV] alpha=0.1, hidden_layer_sizes=(100, 100), learning_rate=invscaling, solver=adam 
[CV]  alpha=0.1, hidden_layer_sizes=(100, 100), learning_rate=invscaling, solver=adam, score=0.5935483870967742, total=   1.2s
[CV] alpha=0.1, hidden_layer_sizes=(100, 100), learning_rate=adaptive, solver=lbfgs 
[CV]  alpha=0.1, hidden_layer_sizes=(100, 100), learning_rate=adaptive, solver=lbfgs, score=0.5192307692307693, total=   1.1s
[CV] alpha=0.1, hidden_

[Parallel(n_jobs=1)]: Done 720 out of 720 | elapsed:  9.5min finished


GridSearchCV(cv=KFold(n_splits=5, random_state=1, shuffle=True),
       error_score='raise-deprecating',
       estimator=MLPClassifier(activation='logistic', alpha=0.0001, batch_size='auto',
       beta_1=0.9, beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=1, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'hidden_layer_sizes': [(50,), (100,), (50, 50), (100, 100)], 'solver': ['lbfgs', 'sgd', 'adam'], 'alpha': [0.0001, 0.001, 0.01, 0.1], 'learning_rate': ['constant', 'invscaling', 'adaptive']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=20)

In [126]:
print('Best accuracy :', best_model.best_score_)
print('Best params   :', best_model.best_params_)

Best accuracy : 0.5943517329910141
Best params   : {'alpha': 0.01, 'hidden_layer_sizes': (100, 100), 'learning_rate': 'constant', 'solver': 'adam'}


In [129]:
best_params = {'alpha': 0.01, 'hidden_layer_sizes': (100, 100), 'learning_rate': 'constant', 'solver': 'adam'}
model = MLPClassifier(random_state=1, activation='logistic', **best_params)
print_cv_result(model, X_Scaled, y)

--- Validation Metrics ---
Accuracy  = 0.594
Precision = 0.362
Recall    = 0.363


# Final Train

In [127]:
best_params = {'alpha': 0.01, 'hidden_layer_sizes': (100, 100), 'learning_rate': 'constant', 'solver': 'adam'}
model = MLPClassifier(random_state=1, activation='logistic', **best_params)
model.fit(X_Scaled, y)

MLPClassifier(activation='logistic', alpha=0.01, batch_size='auto',
       beta_1=0.9, beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100, 100), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=1, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

## Save Best Model

In [128]:
pickle.dump(model, open('{}/taki2_best_model.pkl'.format(MODEL_PATH), 'wb'))

## Reference
1. https://en.wikipedia.org/wiki/Feature_scaling